In [14]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import WebBaseLoader

load_dotenv()

# Correct way to retrieve and print the environment variable
groq_api_key = os.getenv("GROQ_API_KEY")
user_agent = os.getenv("USER_AGENT")
   
llm = ChatGroq(
    temperature=0, 
    groq_api_key=groq_api_key, 
    model_name="llama-3.3-70b-versatile"
    )

response = llm.invoke("The first person to land on moon was ...")
print(response.content)


# if not user_agent:
#     print("USER_AGENT environment variable not set. Using a default.")
#     user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"

try:
    loader = WebBaseLoader(  "https://careers.nike.com/software-engineer/job/R-50041")
    #loader.requests_kwargs = {'verify':False}
    page_data = loader.load().pop().page_content
    print(page_data)


except Exception as e:
    print(f"An error occurred: {e}")

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.





















Software Engineer










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer',
 'experience': '2 years',
 'skills': ['Big data and data lake systems',
  'AWS Cloud infrastructure',
  'Linux sys admin',
  'Docker containerization',
  'Terraform automation',
  'OKTA/IAM integration',
  'Jenkins pipelines',
  'Networking architectures',
  'Kubernetes administration',
  'Python coding',
  'Monitoring tools New relic, Splunk, Signalfx',
  'CM tools such as Puppet, chef, ansible'],
 'description': 'Develop, configure, and test programs, systems and solutions in order to meet defined digital product specifications and directions. Perform technical development of digital products that meets the consumer needs as defined by product managers and aligns with architectural standards. Assess a well-defined problem and support the development of a technical solution that meets the needs of the business and aligns with architectural standards.'}

In [17]:
type(json_res)

dict

In [18]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])


In [21]:
job = json_res
job['skills']

['Big data and data lake systems',
 'AWS Cloud infrastructure',
 'Linux sys admin',
 'Docker containerization',
 'Terraform automation',
 'OKTA/IAM integration',
 'Jenkins pipelines',
 'Networking architectures',
 'Kubernetes administration',
 'Python coding',
 'Monitoring tools New relic, Splunk, Signalfx',
 'CM tools such as Puppet, chef, ansible']

In [22]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],

In [24]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Abhinav Dikshit, a business development executive at Infosys Technology Limited. Infosys is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Infosys's portfolio: {link_list}
        Remember you are Abhinav Dikshit, BDE at Infosys. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Your Business Needs

Dear Hiring Manager,

I came across your job description for a Software Engineer, and I'm excited to introduce you to Infosys Technology Limited, a leading AI & Software Consulting company. Our team of experts has extensive experience in developing, configuring, and testing programs, systems, and solutions that meet defined digital product specifications and directions.

With our expertise in Big data and data lake systems, AWS Cloud infrastructure, Linux sys admin, Docker containerization, Terraform automation, OKTA/IAM integration, Jenkins pipelines, Networking architectures, Kubernetes administration, Python coding, and Monitoring tools like New Relic, Splunk, and Signalfx, we are confident that we can fulfill your requirements. Our team is well-versed in CM tools such as Puppet, Chef, and Ansible, ensuring seamless integration and automation of your business processes.

At Infosys, we have a proven track record